In [8]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull
from skimage import measure

In [9]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [10]:
name_project = 'project/output_FJSJ_17-01/'               # Harmon server
name_project

'project/output_FJSJ_17-01/'

In [11]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/


In [12]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [13]:
key_subworks = info_basic['key_subworks']
key_subworks

In [ ]:
dir_image = dir_project+info_basic['dir_image']
print('dir_image: ', dir_image)

In [18]:
dir_disp =  'autopick'+info_basic['tag']+'/'
info_basic['dir_disp'] = dir_disp
dir_disp = dir_project+dir_disp
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)
print('dir_disp: ', dir_disp)

dir_disp:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/autopick17-01//


### Pick probes' Phase velocity automatically

In [19]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [20]:
def pick_fundamental(ds):
    global f
    global c
    global f_new
    global c_new
    global dir_ds
    global threshold
    # 归一化
    #ds = ds / np.max(ds)  

    binary_ds = ds > threshold
    label_ds,num_labels = measure.label(binary_ds, connectivity=2, return_num=True)
    region_areas = [np.sum(label_ds==i) for i in range(1,num_labels+1)]
    if num_labels == 0:
        return np.zeros(0),np.zeros(0)
    max_area_label = np.argmax(region_areas) + 1
    largest_cluster = label_ds == max_area_label
    indices = np.argwhere(largest_cluster)

    ff = f_new[indices[:,1]]
    cc = c_new[indices[:,0]]

    return ff,cc

In [21]:
def pick_velocity(key_subwork):
    global f
    global c
    global info_basic
    global flag_f
    global flag_c
    global dir_project
    global dir_disp

    grace = 0.02

    dir_ds = dir_project + info_basic['dir_ds']
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')

    ds_remove = ds['ds_remove'][0]
    f_sample = []
    c_sample = []

    f_new = f[np.logical_and(f>flag_f[0],f < flag_f[1])]
    c_new = c[np.logical_and(c>flag_c[0],c < flag_c[1])]

    # Three samples
    f0 = flag_f[0]
    f1 = (flag_f[0] + flag_f[1])/3
    f2 = flag_f[1]
    c0 = c_new[np.argmax(ds_remove[np.logical_and(c>flag_c[0],c < flag_c[1]),f==find_nearest(f,f0)])]
    c1 = c_new[np.argmax(ds_remove[np.logical_and(c>flag_c[0],c < flag_c[1]),f==find_nearest(f,f1)])]
    c2 = c_new[np.argmax(ds_remove[np.logical_and(c>flag_c[0],c < flag_c[1]),f==find_nearest(f,f2)])]

    ds_new0 = ds_remove[np.logical_and(c>flag_c[0],c < flag_c[1]),:]
    ds_new = ds_new0[:,np.logical_and(f>flag_f[0],f < flag_f[1])]   
    

    for index_f in range(len(f_new)):
        #print(index_f)
        if f_new[index_f] >= f0 and f_new[index_f] < f1:
            slope = (c1-c0)/(f1-f0)
            #index_c = np.where(ds_new[:,index_f] == np.max(ds_new[c_new<c0+slope*(f_new[index_f]-f0)+grace,index_f] ))[0][0]
            index_c = np.argmax(ds_new[c_new<c0+slope*(f_new[index_f]-f0)+grace,index_f])
        else:
            slope = (c2-c1)/(f2-f1)
            index_c = np.argmax(ds_new[c_new<c1+slope*(f_new[index_f]-f1)+grace,index_f])
            #index_c = np.where(ds_new[:,index_f] == np.max(ds_new[c_new[c_new<flag_c[1]*0.7]<c1+slope*(f_new[index_f]-f1)+grace,index_f] ))[0][0]
        f_sample.append(f_new[index_f])
        c_sample.append(c_new[index_c])
    
    c_sample = smooth_data(c_sample, window_size=1)

    f_smooth = np.linspace(min(f_sample), max(f_sample), 40)  # np.linspace 等差数列,从x.min()到x.max()生成300个数，便于后续插值
    c_smooth = make_interp_spline(f_sample, c_sample)(f_smooth)
    # 写入txt文件
    np.savetxt(dir_disp+'autopick_'+key_subwork+'.txt', np.array([f_smooth,c_smooth]).T, fmt='%.2f', delimiter=',')
    print('finish '+ key_subwork)
    #return ds_remove, f_smooth, c_smooth,f0,f1,f2,c0,c1,c2


In [22]:
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])

In [23]:
flag_c = [0.35,0.75]
flag_f = [10,15]
nThreads = 5
c_mean = {}
c_mean_data = np.zeros(len(f))
flag_plot = 0

In [24]:
nThreads = 20
pool = ThreadPoolExecutor(max_workers = nThreads)
for key_subwork in key_subworks:
    pool.submit(pick_velocity,key_subwork)
pool.shutdown()

finish 09-02
finish 21-02
finish 20-02
finish 12-02
finish 16-02
finish 07-02
finish 06-02
finish 14-02
finish 17-02
finish 08-02
finish 18-02
finish 22-02
finish 10-02
finish 13-02
finish 26-02
finish 11-02
finish 27-02
finish 04-02
finish 19-02
finish 25-02
finish 32-02
finish 23-02
finish 38-02
finish 05-02
finish 31-02
finish 36-02
finish 24-02
finish 29-02
finish 35-02
finish 39-02
finish 33-02
finish 41-02
finish 15-02
finish 46-02
finish 45-02
finish 50-02
finish 44-02
finish 37-02
finish 30-02
finish 28-02
finish 48-02
finish 51-02
finish 47-02
finish 42-02
finish 54-02
finish 53-02
finish 43-02
finish 04-03
finish 56-02
finish 40-02
finish 55-02
finish 05-03
finish 49-02finish 52-02

finish 10-03
finish 08-03
finish 34-02
finish 06-03
finish 11-03
finish 57-02
finish 13-03
finish 09-03
finish 17-03
finish 15-03
finish 07-03
finish 19-03
finish 21-03
finish 25-03
finish 18-03
finish 12-03
finish 16-03
finish 29-03
finish 27-03
finish 20-03
finish 26-03
finish 31-03
finish 24-03

In [25]:
def plot_disp(key_subwork):
    global dir_disp
    global f
    global c
    global dir_image_disp

    fig,ax = plt.subplots(1,1,figsize=(8,6))
    dir_ds = dir_project + info_basic['dir_ds']
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]

    title0 = key_subwork
    ax = plotlib.plot_fj(ax,ds_remove,title0,f,c,0,c_map='jet')

    f_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    c_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    ax.scatter(f_pick,c_pick,marker='o',s=10,color='k')
    plt.savefig(dir_image_disp+'autopick_'+key_subwork+'.png',dpi=60)
    plt.close()
    print('finish '+key_subwork)

In [26]:
dir_image_disp = 'image_autopick/'
if not os.path.exists(dir_project+dir_image_disp):
    os.makedirs(dir_project+dir_image_disp)
dir_image_disp = dir_project+dir_image_disp

In [27]:
key_subwork = key_subworks[0]
plot_disp(key_subwork)

finish 04-02
